In [1]:
from tardis.workflows.standard_simulation_solver import StandardSimulationSolver
from tardis.workflows.v_inner_simulation_solver import InnerVelocitySimulationSolver
from tardis.io.configuration.config_reader import Configuration

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [2]:
import numpy as np
from functools import reduce
from astropy import units as u
import numpy.ma as ma

a = np.arange(10) * u.K
b = a[5:]
c = a[4:-1]

b.base, b.newbyteorder(), b, b.base.newbyteorder()


(<Quantity [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.] K>,
 <Quantity [2.561e-320, 3.067e-320, 3.573e-320, 4.079e-320, 4.332e-320] K>,
 <Quantity [5., 6., 7., 8., 9.] K>,
 <Quantity [0.00000e+000, 3.03865e-319, 3.16202e-322, 1.04347e-320,
            2.05531e-320, 2.56124e-320, 3.06716e-320, 3.57308e-320,
            4.07901e-320, 4.33197e-320] K>)

In [3]:
d = ma.masked_array(a, mask=np.zeros(10, dtype=bool))
d, d.data

(d * 2).data


<Quantity [ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18.] K>

In [4]:
def reproject(*arrs):

    masks = []
    for a in arrs:
        base_byteorder = a.base.newbyteorder()
        current_byteorder = a.newbyteorder()
        if hasattr(current_byteorder, 'unit'):
            current_byteorder = current_byteorder.value
        if hasattr(base_byteorder, 'unit'):
            base_byteorder = base_byteorder.value
        position_mask = np.in1d(base_byteorder, current_byteorder)
        masks.append(position_mask)

    joint_mask = reduce(np.ndarray.__and__, masks)
    outputs = []
    for a in arrs:
        a_joined = a.base[joint_mask]
        if not hasattr(a_joined, 'unit') and hasattr(a, 'unit'):
            a_joined = a_joined * a.unit
        outputs.append(a_joined)
    return outputs
reproject(b, c)

[<Quantity [5., 6., 7., 8.] K>, <Quantity [5., 6., 7., 8.] K>]

In [5]:
reproject(a, b)

[<Quantity [5., 6., 7., 8., 9.] K>, <Quantity [5., 6., 7., 8., 9.] K>]

In [6]:
config = Configuration.from_yaml('../tardis_example.yml')


In [7]:
solver = InnerVelocitySimulationSolver(config)

Number of density points larger than number of shells. Assuming inner point irrelevant


In [8]:
solver.solve()

/home/jobrien/Documents/soft/tardis/tardis/transport/montecarlo/montecarlo_main_loop.py:142: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  vpacket_collection = vpacket_collections[i]


Emitted Luminosity: 1.641184580261326e+42 erg / s
1058871589.2676698 cm / s
{'t_radiative': <Quantity [12625.037612  , 12860.53394613, 12980.79070524, 13215.76366688,
           13363.89878945, 13657.14421301, 13763.63823313, 13804.67011851,
           13958.82452403, 14072.71071308, 14170.37621888, 14284.27251711,
           14254.01748483, 14271.4519333 , 14326.51256447, 14413.81498266,
           14436.60593629, 14298.86614474, 14369.33440387, 14455.37717072,
           14281.61044524, 14315.33110458, 14398.72767221, 14248.36704032,
           14299.58975802, 14131.11483101, 14011.09856031, 14172.95687047,
           14053.30885316, 14104.26525104, 13950.492525  , 14083.66093865,
           13958.00574775, 13835.47644023, 13842.33838291, 13733.92571628,
           13628.17158291, 13524.38615799, 13696.73931507, 13697.48811155,
           13665.86456317, 13544.81069125, 13464.140919  , 13283.55314878,
           13081.62886491, 12956.89458443, 12817.44163039, 12691.15610746,
        

KeyError: 'lines'

In [ ]:
%debug

> /home/jobrien/miniconda3/envs/tardis/lib/python3.12/site-packages/pandas/core/indexes/base.py(3812)get_loc()
   3810             ):
   3811                 raise InvalidIndexError(key)
-> 3812             raise KeyError(key) from err
   3813         except TypeError:
   3814             # If we have a listlike key, _check_indexing_error will raise

ipdb> u
> /home/jobrien/miniconda3/envs/tardis/lib/python3.12/site-packages/pandas/core/frame.py(4102)__getitem__()
   4100             if self.columns.nlevels > 1:
   4101                 return self._getitem_multilevel(key)
-> 4102             indexer = self.columns.get_loc(key)
   4103             if is_integer(indexer):
   4104                 indexer = [indexer]

ipdb> u
> /home/jobrien/Documents/soft/tardis/tardis/plasma/properties/base.py(116)update()
    114                 for i, output in enumerate(self.outputs):
    115                     setattr(self, output, new_values[i])
--> 116         else:
    117             logger.info

ipdb> len(new_values)
29224


In [ ]:
config = Configuration.from_yaml('../tardis_example.yml')
config.model.structure.velocity.start = solver.simulation_state.v_inner_boundary
config.model.structure.velocity.num = solver.simulation_state.no_of_shells
print(solver.simulation_state.v_inner_boundary)
solver2 = StandardSimulationSolver(config)
solver2.solve()

%debug

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
spectrum = solver.spectrum_solver.spectrum_real_packets
spectrum_virtual = solver.spectrum_solver.spectrum_virtual_packets
spectrum_integrated = solver.spectrum_solver.spectrum_integrated

spectrum2 = solver2.spectrum_solver.spectrum_real_packets
spectrum_virtual2 = solver2.spectrum_solver.spectrum_virtual_packets
spectrum_integrated2 = solver2.spectrum_solver.spectrum_integrated

In [ ]:
%matplotlib inline
plt.figure(figsize=(10, 6.5))

#spectrum.plot(label="Normal packets")
spectrum_virtual.plot(label="Virtual packets")
#spectrum_integrated.plot(label='Formal integral')

#spectrum2.plot(label="Normal packets Original")
spectrum_virtual2.plot(label="Virtual packets Original")
#spectrum_integrated2.plot(label='Formal integral Original')

plt.xlim(500, 9000)
plt.title("TARDIS example model spectrum")
plt.xlabel("Wavelength [$\AA$]")
plt.ylabel("Luminosity density [erg/s/$\AA$]")
plt.legend()
plt.show()

In [ ]:
solver.simulation_state.v_inner

In [ ]:
solver2.simulation_state.v_inner

In [ ]:
plt.plot(solver.simulation_state.v_inner, solver.simulation_state.t_radiative, label='Vinner Solver')
plt.plot(solver2.simulation_state.v_inner, solver2.simulation_state.t_radiative, label='Original Solver')

plt.xlabel('v_inner')
plt.ylabel('t_rad')
plt.legend()

In [ ]:
plt.plot(solver.simulation_state.v_inner, solver.simulation_state.dilution_factor, label='Vinner Solver')
plt.plot(solver2.simulation_state.v_inner, solver2.simulation_state.dilution_factor, label='Original Solver')

plt.xlabel('v_inner')
plt.ylabel('w')
plt.legend()

In [ ]:
solver.simulation_state.t_inner, solver2.simulation_state.t_inner

In [ ]:
solver.simulation_state.dilution_factor

In [ ]:
solver2.simulation_state.dilution_factor

In [ ]:
from tardis.workflows.util import get_tau_integ
import numpy as np

tau_integ =  get_tau_integ(
                solver.plasma_solver,
                solver.simulation_state,
            )['rossland']
        

In [ ]:
tau_integ

In [ ]:
plt.plot(solver.simulation_state.v_inner, tau_integ)

In [ ]:
tau_integ2 = get_tau_integ(
                solver2.plasma_solver,
                solver2.simulation_state,
            )['rossland']

In [ ]:
plt.plot(solver2.simulation_state.v_inner, tau_integ2)

In [ ]:
solver.plasma_solver.electron_densities

In [ ]:
transport_state, virtual_packet_energies = solver.solve_montecarlo(
                solver.real_packet_count
            )

In [ ]:
transport_state.radfield_mc_estimators.nu_bar_estimator.shape

In [ ]:
transport_state2, virtual_packet_energies2 = solver2.solve_montecarlo(
                solver.real_packet_count
            )
transport_state2.radfield_mc_estimators.nu_bar_estimator.shape

In [ ]:
transport_state.radfield_mc_estimators.j_estimator.shape, transport_state2.radfield_mc_estimators.j_estimator.shape

In [ ]:
solver.plasma_solver.electron_densities

In [ ]:
solver2.plasma_solver.electron_densities

In [ ]:
plt.plot(solver.simulation_state.v_inner, 
         solver.plasma_solver.electron_densities, 
         label='V_inner Solver')
plt.plot(solver2.simulation_state.v_inner, 
         solver2.plasma_solver.electron_densities, 
         label='Original Solver')
plt.ylabel('electron_densities')

In [ ]:
plt.plot(solver.simulation_state.geometry.v_inner, 
         solver.simulation_state.composition.elemental_number_density.loc[8], 
         label='V_inner Solver')
plt.plot(solver2.simulation_state.geometry.v_inner, 
         solver2.simulation_state.composition.elemental_number_density.loc[8], 
         label='Original Solver')

In [ ]:
solver2.simulation_state.composition.elemental_number_density

In [ ]:
solver.plasma_solver.outputs_dict.keys()

In [ ]:
solver.plasma_solver.outputs_dict['abundance'].name

In [ ]:
solver.simulation_state.abundance